## PI-3 Running SFINCS using the National Water Model 3.0 inputs

Below is a notebook for running SFINCS using National Water Model inputs over a domain of interest

Author: Tadd Bindas

### Step 0: Import dependencies and create national water model data

In [16]:
from pathlib import Path

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import fiona
from shapely.ops import unary_union
import matplotlib.pyplot as plt
import contextily as cx
import networkx as nx
import zarr

import hydromt
from hydromt_sfincs import SfincsModel
from hydromt_sfincs import utils
from hydromt_sfincs import workflows

### Step 1: Define the model:

First, we need to create a SFINCS Model object and some Data Catalogs to point at external data. The SFINCS object saves internal states and a config for mapping to external files

In [7]:
root = Path("/Users/taddbindas/projects/rise/data/SFINCS/ngwpc_data/")
yml_str = f"""
meta:
  root: {root.__str__()}
  
10m_lidar:
  path: HUC6_110701_dem.tiff
  data_type: RasterDataset
  driver: raster
  driver_kwargs:
    chunks:
      x: 6000
      y: 6000
  meta:
    category: topography
    crs: 5070
  rename:
    10m_lidar: elevtn
"""
data_lib = (
    "/Users/taddbindas/projects/rise/data/SFINCS/data_catalogs/10m_huc6_lidar.yml"
)
with open(data_lib, mode="w") as f:
    f.write(yml_str)

In [13]:
data_catalog = hydromt.DataCatalog(data_libs=[data_lib])
sf = SfincsModel(data_libs=[data_lib], root="../data/ngwpc_data", mode="w+")

### Step 2: Set up the bounding boxes around the catchments of interest

Since we want to determine the flooding downstream of the current RFC station, we have to subset our graph to determine where these catchments exist

In [15]:
import sys

current_dir = Path.cwd()
src_path = current_dir.parents[0]
sys.path.append(src_path.__str__())

from src.rise.utils import hydrofabric  # noqa: E402

In [ ]:
# Hardcoded HUC8
huc_8 = "11070103"
base_data_path = Path("/Users/taddbindas/projects/rise/data/NWM")
output_geojson = base_data_path / f"output_{huc_8}.geojson"
huc8_data = base_data_path / "huc8s_sp.gpkg"
gdf = gpd.read_file(huc8_data)
subset  = gdf[gdf["HUC8"] == huc_8]
subset.to_file(output_geojson, driver='GeoJSON')

In [ ]:
file_path = Path("/Users/taddbindas/hydrofabric/v20.1/gpkg/nextgen_11.gpkg")

flowlines = hydrofabric.get_layer(file_path, layer="flowpaths")
nexus = hydrofabric.get_layer(file_path, layer="nexus")
divides = hydrofabric.get_layer(file_path, layer="divides")
flowpath_attributes = hydrofabric.get_layer(file_path, layer="flowpath_attributes")

G = hydrofabric.get_hydrofabric_vpu_graph(nexus, flowlines)

In [ ]:
# Hardcoded start and end nodes
start_node = "nex-2177032"
end_node = "nex-2175887"
path = nx.shortest_path(G, start_node, end_node)
path

In [ ]:
mask_flowlines = flowlines["id"].isin(path) | flowlines["toid"].isin(path)
mask_nexus = nexus["id"].isin(path)
mask_divides = divides["id"].isin(path)
# mask_attributes = flowpath_attributes["id"].isin(path)

_subset_nexus = nexus[mask_nexus]
_subset_flowlines = flowlines[mask_flowlines]
_subset_divides = divides[mask_divides]

In [18]:
merged_polygon = unary_union(_subset_divides['geometry'])
merged_gdf = gpd.GeoDataFrame(geometry=[merged_polygon], crs=gdf.crs)
output_divides = "/Users/taddbindas/projects/rise/data/NWM/flowlines_divides.geojson"
merged_gdf.to_file(output_divides, driver='GeoJSON')

NameError: name '_subset_divides' is not defined

In [ ]:
sf.setup_grid_from_region(
    region = {'geom': 'tmp_ngwpc_data/coffeyville/flowlines_divides.geojson'},
    res= 50,
    rotated=True,
    crs=_subset_divides.crs  # NAD83 / Conus Albers HARDCODED TODO figure out making this cleaner
)
# the input file is automatically updated. Uncomment to displayed below:
print(sf.config)

### Step 3: Load in Elevation Data

We're going to be loading in elevation data from FIM-C

In [ ]:
datasets_dep = [{"elevtn": "10m_lidar", "zmin": 0.001}]

dep = sf.setup_dep(datasets_dep=datasets_dep)

_ = sf.plot_basemap(variable="dep", bmap="sat", zoomlevel=12)

### Step 4: Mask active cells, boundaries, and coasts

We're going to be masking all active cells as cells above -5 meters. This will properly define coastal vs non-coastal boundaries

In [ ]:
# Choosing how to choose you active cells can be based on multiple criteria, here we only specify a minimum elevation of -5 meters
sf.setup_mask_active(zmin=-5, reset_mask=True)

# Make a plot of the mask file
_ = sf.plot_basemap(variable="msk", plot_bounds=True, bmap="sat", zoomlevel=12)

### Step 5: Add output boundaries

We're going to now mask the boundaries of catchments/rivers

In [ ]:
sf.setup_mask_bounds(btype="waterlevel", zmax=-5, reset_bounds=True)

# Here we add outflow cells, only where clicked in shapefile along part of the lateral boundaries
sf.setup_mask_bounds(btype="outflow", include_mask=merged_gdf, reset_bounds=True)

# Make a plot of the mask file
fig, ax = sf.plot_basemap(variable="msk", plot_bounds=True, bmap="sat", zoomlevel=12)
merged_gdf.to_crs(sf.crs).boundary.plot(
    ax=ax, color="k", lw=1, ls="--"
)  # plot the shapefile given by the user as dashed line

### Step 6: Add river inflow data

Using the hydrofabric we can add flowlines to show where rivers are located within the Hydrofabric

In [ ]:
sf.setup_river_inflow(
    rivers=_subset_flowlines, keep_rivers_geom=True
)
print(sf.config)
# Make a plot of model
# note the src points and derived river network
fig, ax = sf.plot_basemap(variable="dep", plot_bounds=False, bmap="sat", zoomlevel=12)

In [ ]:
gdf_riv = sf.geoms["rivers_inflow"].copy()
df_reordered = pd.merge(gdf_riv["id"], flowpath_attributes, on='id', how='left')
gdf_riv["rivwth"] = np.mean(df_reordered["TopWdth"].values) # width [m]

# Assuming depth is 1.5m, TODO come back later
gdf_riv["rivdph"] = 1.5  # depth [m]
gdf_riv["manning"] = df_reordered["n"].tolist()  # manning coefficient [s.m-1/3]
gdf_riv[["geometry", "rivwth", "manning"]]

datasets_riv = [{"centerlines": gdf_riv}]

### Step 7: Add spatially varying roughness

Our roughness data needs to be spatially varying, and can use the flowpath attributes in the hydrofabric

In [ ]:
gdf_riv_buf = gdf_riv.assign(geometry=gdf_riv.geometry.buffer(0.2))
# da_manning = sf.grid.raster.rasterize(gdf_riv_buf, "manning", nodata=np.nan)
da_manning = sf.grid.raster.rasterize(gdf_riv, "manning", nodata=np.nan)

# use the river manning raster in combination with vito land to derive the manning roughness file
# NOTE that we can combine in-memory data with data from the data catalog
datasets_rgh = [{"manning": da_manning}]

# uncomment to plot either the raster or the vector data:
da_manning.plot(vmin=0, x='xc', y='yc', cmap='viridis')

### Step 8: Make subgrid derived tables:
Subgrid derived tables are used to better capture the elevation and roughness of your domain, to either improve your results, or to allow you to run on a courser grid resolution (means faster simulation). For more info about subgrid tables, [click here](https://sfincs.readthedocs.io/en/latest/developments.html#recent-advancements-in-accuracy-subgrid-mode). 

You as user can specify multiple settings about how the subgrid derived tables should be made.

Every single grid cell of the flux grid of the size inp.dx by inp.dy is defined into subgrid pixels (default nr_subgrid_pixels = 20).
For every subgrid pixel the topobathy data is loaded, ideally this consists of high-resolution DEM datasets that you specify as user.

In this example with dx=dy=50m, having nr_subgrid_pixels = 20 means we are loading data onto a 2.5 m subpixel grid
However, the input data of Gebco and Merit_hydro is way coarser, therefore let's set the ratio to 5 for now.

In [ ]:
sf.setup_subgrid(
    datasets_rgh=datasets_rgh,
    datasets_dep=datasets_dep,
    datasets_riv=datasets_riv,
    nr_subgrid_pixels=5,
    write_dep_tif=False,
    write_man_tif=False,
)
_ = sf.plot_basemap(
    variable="subgrid.z_zmin", plot_bounds=False, bmap="sat", zoomlevel=12
)

### Step 9: Add an upstream discharge time-series as forcing (NWM retrospective)

We're going to be pulling inputs from the national water model retrospective using TEEHR

In [ ]:
conus_df = pd.read_parquet("/Users/taddbindas/hydrofabric/v20.1/conus_net.parquet")
_df = conus_df[conus_df["id"].isin(_subset_flowlines["id"].values)]
_feature_ids = _df[~np.isnan(_df["hf_id"])]
feature_ids = _feature_ids["hf_id"].values.astype(int)
mapping = {}
for _hf_id, _id in zip(_feature_ids["hf_id"], _feature_ids["id"]):
    _mapped_features = mapping.get(_id, None)
    if _mapped_features is None:
        mapping[_id] = [_hf_id]
    else:
        mapping[_id].append(_hf_id)

In [ ]:
from teehr.loading.nwm import retrospective_points as nwm_retro
from datetime import datetime
# Define the parameters.
NWM_VERSION = "nwm30"
VARIABLE_NAME = "streamflow"
START_DATE = datetime(2019, 5, 20)
END_DATE = datetime(2019, 5, 28)
LOCATION_IDS = feature_ids

OUTPUT_ROOT = Path("/Users/taddbindas/projects/hydromt_sfincs/examples/tmp_ngwpc_data")
OUTPUT_DIR = Path(OUTPUT_ROOT, "nwm30_retrospective")

In [ ]:
# Fetch and load the data.
nwm_retro.nwm_retro_to_parquet(
    nwm_version=NWM_VERSION,
    variable_name=VARIABLE_NAME,
    start_date=START_DATE,
    end_date=END_DATE,
    location_ids=LOCATION_IDS,
    output_parquet_dir=OUTPUT_DIR
)

In [ ]:
flow_data = pd.read_parquet(OUTPUT_DIR / "20190520_20190528.parquet")
flow_data[['nwm_version', 'location_id']] = flow_data['location_id'].str.split('-', expand=True)
flow_data["location_id"].unique()

In [ ]:
root = zarr.open_group(path="/Users/taddbindas/projects/hydromt_sfincs/examples/tmp_ngwpc_data", mode="w")
flood_root = root.require_group("coffeyville.zarr")
flood_root.array(
    name="value_time",
    data=np.array(flow_data["value_time"].unique()),
    dtype="datetime64[ns]"
)
flood_root.tree()

In [ ]:
for k, v in mapping.items():
    v = np.array([str(int(_v)) for _v in v])
    flow = flow_data[flow_data["location_id"].isin(v)].groupby("value_time")["value"].mean().reset_index()["value"].values
    flood_root.array(
        name=k,
        data=flow,
    )  
 
flood_root.tree()

Determine source point catchment ID numbers for pulling zarr data

In [ ]:
gdf_src = workflows.flwdir.river_source_points(
    gdf_riv=_subset_flowlines,
    gdf_mask=sf.region,
    src_type="inflow",
    buffer=200,
    river_upa=10.0,
    river_len=1e3,
    da_uparea=None,
    reverse_river_geom=False,
    logger=sf.logger,
)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
_subset_flowlines.plot(ax=ax, color='blue', linewidth=0.5, alpha=0.5, label='Flowlines')
gdf_src.plot(ax=ax, color='red', markersize=30, label='Source Points')
try:
    cx.add_basemap(ax, crs=gdf_src.crs.to_string(), source=cx.providers.OpenStreetMap.Mapnik)
except Exception as e:
    print(f"Couldn't add basemap: {e}")
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
buffered_lines = _subset_flowlines.copy()
buffered_lines['geometry'] = _subset_flowlines.geometry.buffer(50)
intersection = gpd.overlay(gdf_src, buffered_lines, how='intersection')

In [ ]:
time = flood_root["value_time"][:]
dis = []
for _id in intersection["id"]:
    dis.append(flood_root[_id][:])
dis = np.array(dis).T

index = sf.forcing["dis"].index
dispd = pd.DataFrame(index=time, columns=index, data=dis)
# # now we call the function setup_discharge_forcing, which adds the discharge forcing to the src points
sf.setup_discharge_forcing(timeseries=dispd)

# # NOTE: the discharge forcing data is now stored in the sf.forcing dictionary
sf.forcing.keys()

### Step 10: Add weirfile:
In SFINCS, a weirfile is often used to explicity account for line-element features such as dikes, dunes or floodwalls. Read more about structures in the [SFINCS manual](https://sfincs.readthedocs.io/en/latest/input_structures.html)

In [ ]:
weir_gdf = gpd.read_file("/Users/taddbindas/projects/hydromt_sfincs/examples/tmp_ngwpc_data/nld_subset_levees.gpkg")
weir_gdf.to_file("/Users/taddbindas/projects/hydromt_sfincs/examples/tmp_ngwpc_data/nld_subset_levees.geojson", driver='GeoJSON')
# In this example specify a 'line' style shapefile for the location of the weir to be added
# NOTE: optional: dz argument - If provided, for weir structures the z value is calculated from the model elevation (dep) plus dz.
sf.setup_structures(
    structures="/Users/taddbindas/projects/hydromt_sfincs/examples/tmp_ngwpc_data/nld_subset_levees.geojson",
    stype="weir",
    dz=None,
)

# NOTE: the observation points are now stored in the sf.geoms dictionary
sf.geoms.keys()

### Step 11: Show and write model settings to disk

In [ ]:
# Use predefined plotting function 'plot_basemap' to show your full SFINCS model setup
_ = sf.plot_basemap(fn_out="basemap.png", bmap="sat", zoomlevel=12)

In [ ]:
sf.write()  # write all
dir_list = os.listdir(sf.root)
print(dir_list)

Now, we can proceed to the `run_sfincs_from_nwm.ipynb` to run and visualize this flood model!